In [1]:
import tensorflow as tf 
from tensorflow import keras
import numpy as np
import keras2onnx
import onnxruntime
print(f"Tensoflow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")
from models import endpoint_in_model
from time import time

Tensoflow Version: 2.3.0
Keras Version: 2.4.0


## Convert keras model to onnx

* Load Kers model
* Convert to onnx format
* Perform inference on onnx model
* Benchmark for time

In [2]:
# Loading keras model

pp_model=tf.keras.models.load_model('/netpool/work/gpu-3/users/malyalasa/New_folder/wandb/run-20210627_124548-3rs0bk0s/files/model-best.h5',compile=False,custom_objects={'CustomMaskLayer':endpoint_in_model.CustomMaskLayer})


In [10]:
# Load test data from numpy array

test_np_array_path = '/netpool/work/gpu-3/users/malyalasa/New_folder/rosbag2numpy/dataset/test_dataset_allscene.npz'

np_data = np.load(test_np_array_path,allow_pickle=True,mmap_mode='r')
np_tst_gridmap=np_data["np_tst_gridmap"][0],
np_tst_grid_org_res=np_data["np_tst_grid_org_res"][0],
np_tst_left_bnd=np_data["np_tst_left_bnd"][0],
np_tst_right_bnd=np_data["np_tst_right_bnd"][0],
np_tst_car_odo=np_data["np_tst_car_odo"][0],
np_tst_init_path=np_data["np_tst_init_path"][0],
list_tst_file_details=np_data["list_tst_file_details"],
np_tst_opt_path=np_data["np_tst_opt_path"]
print(
    f"{np.shape(np_tst_gridmap)}{np.shape(np_tst_grid_org_res)}{np.shape(np_tst_left_bnd)}{np.shape(np_tst_right_bnd)}{np.shape(np_tst_car_odo)}{np.shape(np_tst_init_path)}"
)

np_data.close()

(1, 1536, 1536)(1, 3)(1, 25, 2)(1, 25, 2)(1, 3)(1, 25, 2)


In [11]:
np_tst_gridmap = np.reshape(np_tst_gridmap,newshape=(1536,1536,1))
np_tst_grid_org_res = np.reshape(np_tst_grid_org_res,newshape=(3,))
np_tst_car_odo = np.reshape(np_tst_car_odo,newshape=(3,))
inputs = [np_tst_gridmap.astype(np.float32),
            np_tst_grid_org_res.astype(np.float32), np_tst_left_bnd[0].astype(np.float32),
            np_tst_right_bnd[0].astype(np.float32), np_tst_car_odo.astype(np.float32), 
            np_tst_init_path[0].astype(np.float32)]

inputs = [np.expand_dims(input_p,axis=0) for input_p in inputs]

[print(f"{np.shape(input)}- index {i}") for i,input in enumerate(inputs)]
[print(type(input)) for input in inputs]

(1, 1536, 1536, 1)- index 0
(1, 3)- index 1
(1, 25, 2)- index 2
(1, 25, 2)- index 3
(1, 3)- index 4
(1, 25, 2)- index 5
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


[None, None, None, None, None, None]

In [14]:
#np.shape(np_tst_gridmap[0])
np.savez("/netpool/work/gpu-3/users/malyalasa/New_folder/rosbag2numpy/dataset/sample_gridmap.npz",input_1=inputs[0],Grid_origin_res=inputs[1],
Left_boundary=inputs[2],Right_boundary=inputs[3],Car_loc=inputs[4],initial_path=inputs[5])


In [6]:
# create onnxruntime session
# convert to onnx format

onnx_model = keras2onnx.convert_keras(pp_model, pp_model.name)
content = onnx_model.SerializeToString()
sess = onnxruntime.InferenceSession(content)

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 87 -> 39


In [7]:
x = [np_tst_gridmap,np_tst_grid_org_res,np_tst_left_bnd,np_tst_right_bnd,np_tst_car_odo,np_tst_init_path]


np_tst_gridmap = sess.get_inputs()[0].name
np_tst_grid_org_res = sess.get_inputs()[1].name
np_tst_left_bnd = sess.get_inputs()[2].name
np_tst_right_bnd = sess.get_inputs()[3].name
np_tst_car_odo = sess.get_inputs()[4].name
np_tst_init_path = sess.get_inputs()[5].name

#input_name_X = sess.get_inputs()[6].name

"""
inputs[0] = sess.get_inputs()[0].name
inputs[1] = sess.get_inputs()[1].name
inputs[2] = sess.get_inputs()[2].name
inputs[3] = sess.get_inputs()[3].name
inputs[4] = sess.get_inputs()[4].name
inputs[5] = sess.get_inputs()[5].name

"""


'\ninputs[0] = sess.get_inputs()[0].name\ninputs[1] = sess.get_inputs()[1].name\ninputs[2] = sess.get_inputs()[2].name\ninputs[3] = sess.get_inputs()[3].name\ninputs[4] = sess.get_inputs()[4].name\ninputs[5] = sess.get_inputs()[5].name\n\n'

In [8]:
#x = x if isinstance(x, list) else [x]

#feed = dict([(input.name, x[n]) for n, input in enumerate(sess.get_inputs())])
output_name = sess.get_outputs()[0].name

start = time()
pred_onnx = sess.run([output_name], {np_tst_gridmap:inputs[0],
                            np_tst_grid_org_res:inputs[1],
                            np_tst_left_bnd:inputs[2],
                            np_tst_right_bnd:inputs[3],
                            np_tst_car_odo:inputs[4],
                            np_tst_init_path:inputs[5],})
end = time()                            
print(f"Latency for prediction:{end-start}")

Latency for prediction:0.024233579635620117


In [13]:
""" Print predictions from onnx model """
print(pred_onnx[0][0])

[[ 352.15952 -668.8621 ]
 [ 351.0937  -670.61884]
 [ 349.85403 -672.3463 ]
 [ 348.70724 -674.0468 ]
 [ 348.0208  -675.7516 ]
 [ 347.5078  -676.9351 ]
 [ 347.2672  -677.4874 ]
 [ 347.26413 -677.4262 ]
 [ 347.3713  -677.1948 ]
 [ 347.36798 -677.3059 ]
 [ 347.48407 -677.4504 ]
 [ 347.2786  -677.55176]
 [ 347.27347 -677.3949 ]
 [ 347.40048 -677.50415]
 [ 347.3344  -677.27167]
 [ 347.5138  -677.21344]
 [ 347.4646  -677.01465]
 [ 347.55908 -677.16235]
 [ 347.6316  -677.1131 ]
 [ 347.7598  -677.245  ]
 [ 347.59314 -677.27155]
 [ 347.61435 -677.24207]
 [ 347.53418 -677.0556 ]
 [ 347.46274 -677.2111 ]
 [ 347.4775  -677.29553]]


In [58]:
""" Save Model in ONNX format """

#temp_model_file = 'trained_models/run-20210627_124548-3rs0bk0s_model-best.onnx'
#keras2onnx.save_model(onnx_model, temp_model_file)

In [26]:
# visualize activations

#np.savez("/netpool/work/gpu-3/users/malyalasa/New_folder/rosbag2numpy/dataset/sample_gridmap.npz",input_1=inputs[0],Grid_origin_res=inputs[1],
#Left_boundary=inputs[2],Right_boundary=inputs[3],Car_loc=inputs[4],initial_path=inputs[5])



In [7]:
data = np.load('/netpool/work/gpu-3/users/malyalasa/New_folder/rosbag2numpy/dataset/sample_gridmap.npz')
input_sample = [data[f'{file}'] for file in data.files]

array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]]], dtype=float32)